In [26]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

In [27]:
# Objective Function (Black Box Function)
def target_func(x, y): 
    return  -(20 * (1 - np.exp(-0.2 * np.sqrt(0.5 * (x**2 + y**2)))) - np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(2 * np. pi * y))) + np.exp(1))

In [28]:
# Hyperparamter Space for inputs x and y
space = [
    Real(-35, 35, name='x'),
    Real(-35, 35, name='y')
]

In [29]:
@use_named_args(space)
def objective(**params):
    x = params['x']
    y = params['y']
    z = target_func(x, y)
    return z

# Bayesian Optimization
result = gp_minimize(objective, space, n_calls=50, random_state=0)

# Print the best parameters found
print("Best parameters found:")
print("x:", result.x[0])
print("y:", result.x[1])

# Print the minimum value of the objective function
print("Minimum value of the objective function:", result.fun)

Best parameters found:
x: 33.520674796205455
y: 18.528723000152247
Minimum value of the objective function: -22.25698796416003


Neural Network Model for Hyper Parameter Tuning

In [34]:
# Define the true objective function z = f(x, y)
def obj_func(x, y):
    return -(20 * (1 - np.exp(-0.2 * np.sqrt(0.5 * (x**2 + y**2)))) - np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(2 * np. pi * y))) + np.exp(1))

In [35]:

# Generate synthetic data for training the neural network
def generate_data():
    x = np.random.uniform(-35, 35, 1000)
    y = np.random.uniform(-35, 35, 1000)
    z = obj_func(x, y)
    return np.vstack((x, y)).T, z

# Generate the data
X, z = generate_data()

# Preprocess the data 
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# Define the hyperparameter space
hyp_space  = [
    Integer(1, 5, name='num_layers'), # Num of layers in the network (depth)
    Integer(10, 100, name='num_units'), # Num of neurons in each hidden layer (width)
    Real(0.0001, 0.1, prior='log-uniform', name='learning_rate'), # Steps size at each iteration 
    Real(0.0, 0.5, name='dropout_rate'), # Probability of droping out a neuron
    Integer(10, 100, name='batch_size'), # Num of samples per batch
    Integer(1, 20, name='epochs') # Num of epochs (iterations over the entire dataset) during training
]

In [38]:
# Defining the neural network model
def NN_model(num_layers, num_units, learning_rate, dropout_rate):
    model = Sequential()
    model.add(Dense(num_units, activation='relu', input_shape=(2,)))
    for _ in range(num_layers - 1):
        model.add(Dense(num_units, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

# Define the objective function to minimize
@use_named_args(hyp_space)
def objective(**params):
    model = KerasRegressor(model=NN_model, **params, verbose=0)
    return -np.mean(cross_val_score(model, X, z, cv=3, n_jobs=-1, scoring='neg_mean_squared_error'))

# Perform Bayesian optimization
result = gp_minimize(objective, hyp_space, n_calls=50, random_state=0, acq_func='EI')

# Outpu best hyperparameters from BayesOpt
print("Best hyperparameters:")
print("num_layers:", result.x[0])
print("num_units:", result.x[1])
print("learning_rate:", result.x[2])
print("dropout_rate:", result.x[3])
print("batch_size:", result.x[4])
print("epochs:", result.x[5])

# Train the model with the best hyperparameters on the entire dataset
best_model = KerasRegressor(
    model=NN_model,
    num_layers=result.x[0],
    num_units=result.x[1],
    learning_rate=result.x[2],
    dropout_rate=result.x[3],
    batch_size=result.x[4],
    epochs=result.x[5],
    verbose=1
)

Best hyperparameters:
num_layers: 5
num_units: 100
learning_rate: 0.0031278782792757273
dropout_rate: 0.0
batch_size: 10
epochs: 17


In [40]:
# Spliting data into training and testing sets
X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2, random_state=0)

# Train best model
best_model.fit(X_train, z_train)

# Evaluate best model
mse = best_model.score(X_test, z_test)
print("Model MSE:", mse)

Epoch 1/17


c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 823us/step - loss: 110.6410
Epoch 2/17
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 760us/step - loss: 24.4244
Epoch 3/17
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - loss: 18.9009
Epoch 4/17
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step - loss: 8.8059
Epoch 5/17
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - loss: 4.7949
Epoch 6/17
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - loss: 2.0004
Epoch 7/17
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - loss: 2.4699
Epoch 8/17
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step - loss: 2.1378
Epoch 9/17
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - loss: 1.5223
Epoch 10/17
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - loss: 1.1802
Epoch 11/17
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - loss: 1.0605
Epoch 12/17
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - loss: 0.6725
Epoch 13/17
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - loss: 0.7699
Epoch 14/17
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - loss: 0.6793
Epoch 15/17
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 760us/step - loss: 0.6